# Guest Lecture COMP7230
## *Using Python packages for Linked Data & spatial data*
#### by Nicholas Car, SURROUND Australia Pty Ltd

This Notebook is the resource used to deliver a guest leacture for the [Austalian National University](https://www.anu.edu.au)'s course [COMP7230](https://programsandcourses.anu.edu.au/2020/course/COMP7230): *Introduction to Programming for Data Scientists*

## About the lecturer
**Nicholas Car**:
* PhD in informatics for irrigation
* A former CSIRO informatics researcher
    * worked on integrating environmental data across government / industry
    * developed data standards
* Has worked in operation IT in government
* Now in a private IT consulting company, [SURROUND Australia Pty Ltd](https://surroundaustralia.com) supplying Data Science solutions

Relevant current work:

* building data processing systems for government & industry
* mainly using Python
    * due to its large number of web and data science packages
* maintains the [RDFlib](https://rdflib.net) Python toolkit
    * for processing [RDF](https://en.wikipedia.org/wiki/Resource_Description_Framework)
* co-chairs the [Australian Government Linked Data Working Group](https://www.linked.data.gov.au) with Armin Haller
    * plans for multi-agency data integration
* still developing data standards


## 0. Lecture Outline
1. Notes about this training material
2. Accessing RDF data
3. Parsing RDF data
4. Data 'mash up'
5. Data Display


## 1. Notes about this training material

* This is an Jupyter Notebook - interactive Python scripting
* You will cover Jupyter Notebooks more, later in this course
* Access this material online at...


## 2. Accessing RDF data

* Here we use an online structured dataset, the Geocoded National Address File for Australia
    * Dataset Persistent Identifier: <https://linked.data.gov.au/dataset/gnaf>
    * The above link redirects to the API at <https://gnafld.net>
* GNAF-LD Data is presented according to *Linked Data* principles
    * online
    * in HTML & machine readable form, RDF
    * RDF is a Knowledge Graph: a graph containing data + model
    * each resource is available via a URI
        * e.g. <https://linked.data.gov.au/dataset/gnaf/address/GAACT714845933>

![GAACT714845933](img/GAACT714845933.png)


2.1. Get the Address GAACT714845933 using the *requests* package

In [ ]:
import requests  # NOTE: you must have installed requests first, it's not a standard package
r = requests.get(
    "https://linked.data.gov.au/dataset/gnaf/address/GAACT714845933"
)
print(r.text)

2.2 Get machine-readable data, RDF triples
Use HTTP Content Negotiation
Same URI, different *format* of data

In [ ]:
r = requests.get(
    "https://linked.data.gov.au/dataset/gnaf/address/GAACT714845933",
    headers={"Accept": "application/n-triples"}
)
print(r.text)

2.3 Get machine-readable data, Turtle
Easier to read

In [ ]:
r = requests.get(
    "https://linked.data.gov.au/dataset/gnaf/address/GAACT714845933",
    headers={"Accept": "text/turtle"}
)
print(r.text)

## 3. Parsing RDF data

Import the RDFlib library for manipulating RDF data
Add some namespaces to shorten URIs

In [ ]:
import rdflib
from rdflib.namespace import RDF, RDFS
GNAF = rdflib.Namespace("http://linked.data.gov.au/def/gnaf#")
ADDR = rdflib.Namespace("http://linked.data.gov.au/dataset/gnaf/address/")
GEO = rdflib.Namespace("http://www.opengis.net/ont/geosparql#")
print(GEO)

Create a graph and add the namespaces to it

In [ ]:
g = rdflib.Graph()
g.bind("gnaf", GNAF)
g.bind("addr", ADDR)
g.bind("geo", GEO)
print(g.serialize(format="text/turtle").decode())

Parse in the machine-readable data from the GNAF-LD
Print the length of the graph to check

In [ ]:
r = requests.get(
    "https://linked.data.gov.au/dataset/gnaf/address/GAACT714845933",
    headers={"Accept": "text/turtle"}
)
g.parse(data=r.text, format="text/turtle")

Print graph length (no. of triples)

In [ ]:
print(len(g))

Print graph content, in Turtle

In [ ]:
print(g.serialize(format="text/turtle").decode())

### 3.1 Getting multi-address data:
3.1.1. Retrieve an index of 10 addresses, in RDF
3.1.2. For each address in the index, get each Address' data
* use paging URI: <https://linked.data.gov.au/dataset/gnaf/address/?page=1>
3.1.3. Get only the street address and map coordinates

#### 3.1.1. Retrieve index

In [ ]:
# clear the graph
g = rdflib.Graph()

r = requests.get(
    "https://linked.data.gov.au/dataset/gnaf/address/?page=1",
    headers={"Accept": "text/turtle"}
)
g.parse(data=r.text, format="text/turtle")
print(len(g))

#### 3.1.2. Parse in each address' data

In [ ]:
for s, p, o in g.triples((None, RDF.type, GNAF.Address)):
    print(s.split("/")[-1])
    r = requests.get(
        str(s),
        headers={"Accept": "text/turtle"}
    )
    g.parse(data=r.text, format="turtle")
    print(len(g))

#### 3.1.3. Extract (& print) street address text & coordinates
(CSV)

In [ ]:
addresses_tsv = "GNAF ID\tAddress\tCoordinates\n"
for s, p, o in g.triples((None, RDF.type, GNAF.Address)):
    for s2, p2, o2 in g.triples((s, RDFS.comment, None)):
        txt = str(o2)
    for s2, p2, o2 in g.triples((s, GEO.hasGeometry, None)):
        for s3, p3, o3 in g.triples((o2, GEO.asWKT, None)):
            coords = str(o3).replace("<http://www.opengis.net/def/crs/EPSG/0/4283> ", "")

    addresses_tsv += "{}\t{}\t{}\n".format(str(s).split("/")[-1], txt, coords)

print(addresses_tsv)

#### 3.1.4. Convert CSV data to PANDAS DataFrame
(CSV)

In [ ]:
import pandas
from io import StringIO
s = StringIO(addresses_tsv)
df1 = pandas.read_csv(s, sep="\t")
print(df1)


#### 3.1.5. SPARQL querying RDF data
A graph query, similar to a database SQL query, can traverse the graph and retrieve the same details as the multiple
loops and Python code above in 3.1.3.

In [ ]:
q = """
SELECT ?id ?addr ?coords
WHERE {
    ?uri a gnaf:Address ;
         rdfs:comment ?addr .

    ?uri geo:hasGeometry/geo:asWKT ?coords_dirty .

    BIND (STRAFTER(STR(?uri), "address/") AS ?id)
    BIND (STRAFTER(STR(?coords_dirty), "4283> ") AS ?coords)
}
ORDER BY ?id
"""
for r in g.query(q):
    print("{}, {}, {}".format(r["id"], r["addr"], r["coords"]))

## 4. Data 'mash up'
Add some fake data to the GNAF data - people count per address

In [ ]:
import pandas
df2 = pandas.read_csv('fake_data.csv')
print(df2)

Merge DataFrames

In [ ]:
df3 = pandas.merge(df1, df2)
print(df3.head())

## 5. Spatial Data Display

#### 5.1 Import shapely tools

In [ ]:
import shapely.wkt
from shapely.geometry import MultiPoint
import json

#### 5.2 Convert WKT to GeoJSON

In [ ]:
points_list = []

for s, p, o in g.triples((None, RDF.type, GNAF.Address)):
    for s2, p2, o2 in g.triples((s, GEO.hasGeometry, None)):
        for s3, p3, o3 in g.triples((o2, GEO.asWKT, None)):
            g1 = shapely.wkt.loads(str(o3).replace("<http://www.opengis.net/def/crs/EPSG/0/4283> ", ""))
            points_list.append(g1)

mp = MultiPoint(points=points_list)

geojson = shapely.geometry.mapping(mp)
print(json.dumps(geojson, indent=4))

*Show results in QGIS*

## License
All the content in this repository is licensed under the [CC BY 4.0 license](https://creativecommons.org/licenses/by/4.0/). Basically, you can:

* copy and redistribute the material in any medium or format
* remix, transform, and build upon the material for any purpose, even commercially

You just need to:

* give appropriate credit, provide a link to the license, and indicate if changes were made
* not apply legal terms or technological measures that legally restrict others from doing anything the license permits

## Contact Information
**Dr Nicholas J. Car**<br />
*Data Systems Architect*<br />
[SURROUND Australia Pty Ltd](https://surroundaustralia.com)<br />
<nicholas.car@surroundaustralia.com><br />
GitHub: [nicholascar](https://github.com/nicholascar)<br />
ORCID: <https://orcid.org/0000-0002-8742-7730><br />